In [1]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, TimeDistributed, Conv2D, MaxPooling1D
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import Adam, Nadam

%matplotlib

/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Using matplotlib backend: Qt5Agg


## Import Tick Data and Create 5min RTH Bars

In [2]:
tick_data = pd.read_feather('../data/processed/ES_tick.feather')
tick_data = tick_data[tick_data['date'] > '2017-07-29']
#Create Index from date column
tick_data.index = tick_data['date']
tick_data.drop(labels=['date'],axis=1,inplace=True)
tick_data.tail()

,last,bid,ask,volume
date,,,,
2018-01-25 19:54:52.612000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:52.615000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:55.332000-05:00,2844.0,2843.75,2844.0,2


In [3]:
#Resample to get 5min bars
five_min_data = pd.DataFrame(
    tick_data['last'].resample('5Min', loffset=datetime.timedelta(minutes=5)).ohlc())

import pandas_market_calendars as mcal
#We hack the NYSE Calendar extending the close until 4:15
class CMERTHCalendar(mcal.exchange_calendar_nyse.NYSEExchangeCalendar):
    @property
    def close_time(self):
        return datetime.time(16, 15)
    
#Create RTH Calendar
nyse = CMERTHCalendar()
schedule = nyse.schedule(start_date=five_min_data.index.min(), 
                         end_date=five_min_data.index.max())

#Filter out those bars that occur during RTH
five_min_data['dates'] = pd.to_datetime(five_min_data.index.to_datetime().date)
five_min_data['valid_date'] = five_min_data['dates'].isin(schedule.index)
five_min_data['valid_time'] = False
during_rth = five_min_data['valid_date'] & \
        (five_min_data.index > schedule.loc[five_min_data['dates'],'market_open']) & \
        (five_min_data.index <= schedule.loc[five_min_data['dates'],'market_close'])
five_min_data.loc[during_rth, 'valid_time'] = True
five_min_data = five_min_data[five_min_data['valid_time'] == True]
five_min_data.drop(['dates','valid_date','valid_time'], axis=1, inplace=True)

#Add ema
five_min_data['ema'] = five_min_data['close'].ewm(span=20, min_periods=20).mean()

#Reset index
five_min_data.reset_index(inplace=True)

five_min_data[81:].head()

,date,open,high,low,close,ema
81,2017-08-01 09:35:00-04:00,2475.50,2476.00,2472.50,2473.50,2470.908594
82,2017-08-01 09:40:00-04:00,2473.50,2474.00,2471.50,2472.50,2471.060194
83,2017-08-01 09:45:00-04:00,2472.25,2473.25,2471.75,2473.00,2471.244978
84,2017-08-01 09:50:00-04:00,2473.00,2473.25,2472.00,2472.75,2471.388343
85,2017-08-01 09:55:00-04:00,2472.75,2473.00,2471.25,2471.25,2471.375165


In [83]:
#Add column for number of seconds elapsed in trading day
five_min_data['sec'] = (five_min_data['date'].values 
                        - five_min_data['date'].values.astype('datetime64[D]')) / np.timedelta64(1,'s')

#Calculate sin & cos time
#24hr time is a cyclical continuous feature
seconds_in_day = 24*60*60
five_min_data['sin_time'] = np.sin(2*np.pi*five_min_data['sec']/seconds_in_day)
five_min_data['cos_time'] = np.cos(2*np.pi*five_min_data['sec']/seconds_in_day)

five_min_data.drop('sec', axis=1, inplace=True)
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727


## Create Test / Train Datasets

In [2]:
five_min_data = pd.read_feather('../data/processed/ES_TFCnn.feather')
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311,-104,96
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308,-104,96
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873,-104,96
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011,-104,96
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727,-104,96


In [44]:
five_min_data = pd.read_hdf('../data/processed/store.h5', key='cnn_data')
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104


In [23]:
data[(data['stc']<0)].shape[0], data[(data['stc']>0)].shape[0]

(138960, 114266)

In [46]:
data = five_min_data[81:]

openp = data['open'].tolist()
highp = data['high'].tolist()
lowp = data['low'].tolist()
closep = data['close'].tolist()
emap = data['ema'].tolist()
sin_time = data['sin_time'].tolist()
cos_time = data['cos_time'].tolist()
btc = data['btc'].tolist()
stc = data['stc'].tolist()

In [47]:
mean = data.mean(axis=0)
std = data.std(axis=0)

In [52]:
WINDOW = 2 #Number of bars in a trading day
EMB_SIZE = 7
STEP = 1
FORECAST = 1

X, Y = [], []
for i in range(0, len(data), STEP):
    try:
        o = openp[i:i+WINDOW]
        h = highp[i:i+WINDOW]
        l = lowp[i:i+WINDOW]
        c = closep[i:i+WINDOW]
        e = emap[i:i+WINDOW]
        ct = cos_time[i:i+WINDOW]
        st = sin_time[i:i+WINDOW]

        o = (np.array(o) - mean['open']) / std['open']
        h = (np.array(h) - mean['high']) / std['high']
        l = (np.array(l) - mean['low']) / std['low']
        c = (np.array(c) - mean['close']) / std['close']
        e = (np.array(e) - mean['ema']) / std['ema']
        
        #o = np.array(o) / c[-1] 
        #h = np.array(h) / c[-1]
        #l = np.array(l) / c[-1]
        #e = np.array(e) / c[-1]
        #c = np.array(c) / c[-1]

        x_i = closep[i:i+WINDOW]
        y_i = closep[(i+WINDOW-1)+FORECAST]  

        last_close = x_i[-1]
        next_close = y_i

        if btc[i+WINDOW-1] > 0:
            y_i = [1, 0]
        else:
            y_i = [0, 1]
        
        x_i = np.column_stack((o, h, l, c, e, ct, st))

    except Exception as e:
        #e.throw()
        break

    X.append(x_i)
    Y.append(y_i)

In [53]:
# Let's split into train and test sets
# Train Set will be from 8/1/17 through 12/31/17, Test Set 1/1/17 - 1/25/17
p = 8547 #Manual split for now
p = int(len(X) * 0.9)
X, Y = np.array(X), np.array(Y)
X_train = X[0:p]
Y_train = Y[0:p]
X_test = X[p:]
Y_test = Y[p:]

#We may want to shuffle the training data -- will look into this later
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#X_train, Y_train = shuffle_in_unison(X_train, Y_train)

# Not sure why this is needed, but we apply it anyway
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1], EMB_SIZE))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1], EMB_SIZE))
X_train[-1]

array([[[ 1.45397935,  1.45203134,  1.4513892 ,  1.45093238,
          1.44904328, -0.51877326, -0.85491187],
        [ 1.45093051,  1.45304762,  1.45291365,  1.45448936,
          1.44958962, -0.5       , -0.8660254 ]]])

## Train CNN Model

In [52]:
model = Sequential()

model.add(
    TimeDistributed(
        Conv2D(32, (7, 7), padding='same', strides=2),
        input_shape=(None, 540, 960, 2)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_53 (TimeDis (None, None, 270, 480, 32 4736      
Total params: 4,736
Trainable params: 4,736
Non-trainable params: 0
_________________________________________________________________


In [59]:
model = Sequential()
model.add(TimeDistributed(Convolution1D(filters=16,
                                        kernel_size=4,
                                        padding='same'),
                          input_shape = (None, WINDOW, EMB_SIZE)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Convolution1D(filters=8,
                        kernel_size=4,
                        padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Flatten()))

model.add(TimeDistributed((Dense(32))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))

model.add(LSTM(32, dropout=0, stateful=False))
model.add(Dropout(0))

model.add(Dense(2))
model.add(Activation('softmax'))
print(model.summary())

ValueError: If a RNN is stateful, it needs to know its batch size. Specify the batch size of your input tensors: 
- If using a Sequential model, specify the batch size by passing a `batch_input_shape` argument to your first layer.
- If using the functional API, specify the batch size by passing a `batch_shape` argument to your Input layer.

In [175]:
model = Sequential()
model.add(Convolution1D(input_shape = (WINDOW, EMB_SIZE),
                        filters=16,
                        kernel_size=4,
                        padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
model.add(Dropout(0.5))

model.add(Convolution1D(filters=8,
                        kernel_size=4,
                        padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(32))
model.add(BatchNormalization())
model.add(LeakyReLU())


model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_65 (Conv1D)           (None, 64, 16)            464       
_________________________________________________________________
batch_normalization_94 (Batc (None, 64, 16)            64        
_________________________________________________________________
leaky_re_lu_97 (LeakyReLU)   (None, 64, 16)            0         
_________________________________________________________________
dropout_65 (Dropout)         (None, 64, 16)            0         
_________________________________________________________________
conv1d_66 (Conv1D)           (None, 64, 8)             520       
_________________________________________________________________
batch_normalization_95 (Batc (None, 64, 8)             32        
_________________________________________________________________
leaky_re_lu_98 (LeakyReLU)   (None, 64, 8)             0         
__________

In [55]:
opt = Nadam(lr=0.0001)

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=30, min_lr=0.000001, verbose=1)
checkpointer = ModelCheckpoint(filepath="model.hdf5", verbose=1, save_best_only=True)


model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, 
          epochs = 25, 
          batch_size = 128, 
          verbose=1, 
          validation_data=(X_test, Y_test),
          callbacks=[reduce_lr, checkpointer],
          shuffle='batch')

Train on 227901 samples, validate on 25323 samples
Epoch 1/25
227901/227901 [==============================] - 10s 46us/step - loss: 0.6938 - acc: 0.5066 - val_loss: 0.6935 - val_acc: 0.4989

Epoch 00001: val_loss improved from inf to 0.69354, saving model to model.hdf5
Epoch 2/25
227901/227901 [==============================] - 8s 36us/step - loss: 0.6927 - acc: 0.5112 - val_loss: 0.6950 - val_acc: 0.4988

Epoch 00002: val_loss did not improve from 0.69354
Epoch 3/25
227901/227901 [==============================] - 8s 36us/step - loss: 0.6912 - acc: 0.5260 - val_loss: 0.7021 - val_acc: 0.5075

Epoch 00003: val_loss did not improve from 0.69354
Epoch 4/25
227901/227901 [==============================] - 9s 38us/step - loss: 0.6885 - acc: 0.5402 - val_loss: 0.7050 - val_acc: 0.5075

Epoch 00004: val_loss did not improve from 0.69354
Epoch 5/25
227901/227901 [==============================] - 8s 37us/step - loss: 0.6845 - acc: 0.5553 - val_loss: 0.7001 - val_acc: 0.5075

Epoch 00005: val

In [56]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [57]:
from sklearn.metrics import confusion_matrix

model.load_weights("model.hdf5")
pred = model.predict(np.array(X_test), batch_size=128)

C = confusion_matrix([np.argmax(y) for y in Y_test], [np.argmax(y) for y in pred])

print (C / C.astype(np.float).sum(axis=1)[:, None])

[[0.26659718 0.73340282]
 [0.27569839 0.72430161]]


In [58]:
C

array([[3325, 9147],
       [3543, 9308]])

In [58]:
sum([np.argmax(y) for y in Y_test])

976

In [42]:
df = pd.DataFrame(np.concatenate((pred, Y_test), axis=1))
df[(df[0]>.6)&(df[2]==1)].shape
df[(df[0]>.6)].shape

#df.shape
#df.sort_values(0,axis=0,ascending=False).head(20)

(274, 4)

In [174]:
C

array([[812,   0],
       [501,   0]])

In [45]:
C / C.astype(np.float).sum(axis=1)[:, None]

array([[0.83673469, 0.15855573, 0.00470958],
       [0.78915663, 0.20481928, 0.0060241 ],
       [0.86335404, 0.13043478, 0.00621118]])

In [69]:
probs = Y_train.sum(axis=0) / Y_train.shape[0]
probs

array([0.55539956, 0.44460044])

In [68]:
pred

array([[0.4436597 , 0.5563404 ],
       [0.5040323 , 0.49596766],
       [0.5359964 , 0.4640036 ],
       ...,
       [0.57311577, 0.4268842 ],
       [0.5362818 , 0.46371824],
       [0.5139088 , 0.48609126]], dtype=float32)

In [120]:
s = np.random.binomial(1, probs[1], pred.shape[0])
s

array([1, 0, 1, ..., 0, 0, 1])

In [121]:
C1 = confusion_matrix([np.argmax(y) for y in Y_test], s)
print (C1 / C1.astype(np.float).sum(axis=1)[:, None])

[[0.58421851 0.41578149]
 [0.59026688 0.40973312]]


In [122]:
([np.argmax(y) for y in Y_test] == s).sum() / pred.shape[0]

0.4984567901234568